In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from prod_details import product_demand_per_dc, handling_out_costs_per_dc
from data import as_is_dc

In [5]:
# Load the data
warehousing_handlingOut_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling Out')
warehousing_handlingIn_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling In')
warehousing_storageCost = pd.read_excel('Warehousing.xlsx', sheet_name='Storage')
demand_data = pd.read_excel('Demand Forecast.xlsx')

warehousing_handlingOut_data.index = warehousing_handlingOut_data.DC
warehousing_handlingIn_data.index = warehousing_handlingOut_data.DC
warehousing_storageCost.index = warehousing_storageCost.DC
demand_data.index = demand_data.state

In [6]:
# Can be deleted if it is not too necessary
dc_product_demand = product_demand_per_dc(as_is_dc, demand_data)
dc_product_demand

{'NY': {'blender': 226319,
  'swing': 6468,
  'chair': 161655,
  'scooter': 3233,
  'skiprope': 969935},
 'ND': {'blender': 31949,
  'swing': 913,
  'chair': 22820,
  'scooter': 22832,
  'skiprope': 136921},
 'IL': {'blender': 271812,
  'swing': 7766,
  'chair': 194150,
  'scooter': 3884,
  'skiprope': 1164904},
 'TN': {'blender': 216583,
  'swing': 6188,
  'chair': 154702,
  'scooter': 3095,
  'skiprope': 928213},
 'WA': {'blender': 51437,
  'swing': 1470,
  'chair': 36741,
  'scooter': 735,
  'skiprope': 220442},
 'TX': {'blender': 190768,
  'swing': 5451,
  'chair': 136263,
  'scooter': 2725,
  'skiprope': 817575},
 'UT': {'blender': 179502,
  'swing': 5129,
  'chair': 128215,
  'scooter': 2564,
  'skiprope': 769295}}

In [7]:
products = ['blender', 'swing', 'chair', 'scooter', 'skiprope']
volume_perBox = [0.0070, 0.0045, 0.0098, 0.0200, 0.0055]
dc_product_demand = product_demand_per_dc(as_is_dc, demand_data)

volume_per_dc = {}

for state, products_dict in dc_product_demand.items():
    volume_per_dc[state] = {}
    for i, product in enumerate(products):
        num_boxes = products_dict[product]
        volume = volume_perBox[i]
        volume_per_dc[state][product] = np.ceil(num_boxes * volume)

print(volume_per_dc)

{'NY': {'blender': 1585.0, 'swing': 30.0, 'chair': 1585.0, 'scooter': 65.0, 'skiprope': 5335.0}, 'ND': {'blender': 224.0, 'swing': 5.0, 'chair': 224.0, 'scooter': 457.0, 'skiprope': 754.0}, 'IL': {'blender': 1903.0, 'swing': 35.0, 'chair': 1903.0, 'scooter': 78.0, 'skiprope': 6407.0}, 'TN': {'blender': 1517.0, 'swing': 28.0, 'chair': 1517.0, 'scooter': 62.0, 'skiprope': 5106.0}, 'WA': {'blender': 361.0, 'swing': 7.0, 'chair': 361.0, 'scooter': 15.0, 'skiprope': 1213.0}, 'TX': {'blender': 1336.0, 'swing': 25.0, 'chair': 1336.0, 'scooter': 55.0, 'skiprope': 4497.0}, 'UT': {'blender': 1257.0, 'swing': 24.0, 'chair': 1257.0, 'scooter': 52.0, 'skiprope': 4232.0}}


In [8]:
# Storage Costs (per cubic meter per month for each state)


total_storage_costs = {}
# Calculate the total costs for each state
for state, items in volume_per_dc.items():
    total_sum = sum(items.values())
    if state in warehousing_storageCost.index:
        cost_per_m3 = warehousing_storageCost.loc[state, 'storage_costs_m3_month']
        total_storage_costs[state] = total_sum * cost_per_m3

total_warehousing_costs = sum(total_storage_costs.values())
print(total_warehousing_costs)

1224666.0
